# Predicting energies efficiently and on mixed-sites

In [136]:
import xgboost as xgb
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from xgboost import XGBRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import time
import matplotlib.pyplot as plt

In [137]:
#Load H binding energy prediction model
H_DFT_model = xgb.Booster({'nthread': 8})
H_DFT_model.load_model("../models/"+"H_DFT.model")

#Load COOH binding energy prediction model
COOH_DFT_model = xgb.Booster({'nthread': 8})
COOH_DFT_model.load_model("../models/"+"COOH_DFT.model")

#Load mixed site energy prediction model
mixed_DFT_model = xgb.Booster({'nthread': 8})
mixed_DFT_model.load_model("../models/"+"COOH_H.model")

models = {"H": H_DFT_model, "COOH": COOH_DFT_model, "mixed": mixed_DFT_model}

In [138]:
print(models["H"].num_features())
print(models["COOH"].num_features())
print(models["mixed"].num_features())
print(models["mixed"].attributes())

55
20
75
{'best_iteration': '18', 'best_ntree_limit': '19', 'best_score': '0.150929', 'scikit_learn': '{"n_estimators": 500, "objective": "reg:squarederror", "max_depth": 5, "learning_rate": 0.1, "verbosity": null, "booster": null, "tree_method": null, "gamma": null, "min_child_weight": null, "max_delta_step": null, "subsample": null, "colsample_bytree": null, "colsample_bylevel": null, "colsample_bynode": null, "reg_alpha": null, "reg_lambda": null, "scale_pos_weight": null, "base_score": null, "missing": NaN, "num_parallel_tree": null, "random_state": null, "n_jobs": 8, "monotone_constraints": null, "interaction_constraints": null, "importance_type": null, "gpu_id": null, "validate_parameters": null, "predictor": null, "enable_categorical": false, "kwargs": {"use_label_encoder": false}, "evals_result_": {"validation_0": {"mae": [0.593315, 0.537968, 0.487932, 0.442318, 0.401622, 0.365934, 0.332539, 0.302993, 0.276135, 0.250841, 0.229254, 0.20982, 0.191445, 0.174601, 0.159716, 0.146139

### Compare time to make predictions of COOH and H on a 100 by 100 surface with the new way to predict energies

#### Old method:

In [139]:
dim_x, dim_y, split = 100, 100, [1, 1, 1, 1, 1]

# Note the stochiometry - used for plotting
stochiometry = dict(zip(metals, np.array(split)/np.sum(split)))
# Make the surface (empty surface)
surface = initialize_surface(dim_x, dim_y, metals, split)

# Predict energies on all sites for both adsorbates
surface = precompute_binding_energies_TQDM(surface, dim_x, dim_y, models, predict_G)


Predicting all ΔG:   0%|          | 0/10000 [00:00<?, ?it/s]

#### New method:

In [141]:
dim_x, dim_y, split = 1000, 100, [1, 1, 1, 1, 1]

# Note the stochiometry - used for plotting
stochiometry = dict(zip(metals, np.array(split)/np.sum(split)))
# Make the surface (empty surface)
surface = initialize_surface(dim_x, dim_y, metals, split)

# Predict energies on all sites for both adsorbates
surface = precompute_binding_energies_SPEED(surface, dim_x, dim_y, models)

### With mixed-site energies

In [133]:
dim_x, dim_y, split = 100, 100, [1, 1, 1, 1, 1]

# Note the stochiometry - used for plotting
stochiometry = dict(zip(metals, np.array(split)/np.sum(split)))
# Make the surface (empty surface)
surface = initialize_surface(dim_x, dim_y, metals, split)

# Predict energies on all sites for both adsorbates
surface = precompute_binding_energies_SPEED(surface, dim_x, dim_y, models)

# Predict energies on the mixed-sites
surface = predict_mixed_energies(surface, dim_x, dim_y, models)

In [109]:
# Efter man har fundet den mixed energi, så kan man både få "COOH givet H" energien ud og "H givet COOH" energien

$$ \Delta G(*COOH(*H)) = \Delta G(*COOH + *H) -  \Delta G(*H) $$

$$ \Delta G(*H(*COOH)) = \Delta G(*COOH + *H) -  \Delta G(*COOH) $$

**Down**

COOH(*H) is equal to the mixed energy subtracted the H energy with y = -1

H(*COOH) is equal to the mixed energy subtracted the COOH energy with y = +1

**Up right**

COOH(*H) is equal to the mixed energy subtracted the H energy with the *same index*

H(*COOH) is equal to the mixed energy subtracted the COOH energy with the *same index*

**Up left**

COOH(*H) is equal to the mixed energy subtracted the H energy with x = -1

H(*COOH) is equal to the mixed energy subtracted the COOH energy with x = +1

I up right situationen skal man trække den rene H energi fra som har samme index

Med up left skulle man måske roll'e H energierne, så deres x-værdi bliver rykket én ned.